测试三重过滤分析系统报告

In [1]:
import backtrader as bt
import akshare as ak
import os
import logging
from logging.handlers import TimedRotatingFileHandler
from datetime import datetime
from my_backtrader.tts import TripleScreenTradingSystem

In [2]:
def init_logging():
    """全局日志配置（在策略初始化前调用）"""
    log_format = '%(asctime)s [%(levelname)s] %(message)s'
    log_file = 'logs/futures.log'
    
    # 创建日志目录
    os.makedirs('logs', exist_ok=True)
    
    # 主日志配置
    logging.basicConfig(
        level=logging.INFO,
        format=log_format,
        handlers=[
            TimedRotatingFileHandler(
                log_file, 
                when='D',  # 按天切割
                backupCount=7,
                encoding='utf-8'
            ),
            logging.StreamHandler()
        ]
    )
    
init_logging()

In [3]:
import pandas as pd


def futures_main_weekly_sina(
    symbol: str = "V0",
    start_date: str = "19900101",
    end_date: str = "22220101",
) -> pd.DataFrame:
    """
    新浪财经-期货-主力连续周线数据
    基于日线数据聚合计算，或直接调用周线接口
    """
    # 1. 获取日线数据后聚合为周线（通用方法）
    daily_df = ak.futures_main_sina(symbol=symbol, start_date=start_date, end_date=end_date)
    daily_df["日期"] = pd.to_datetime(daily_df["日期"])
    daily_df.set_index("日期", inplace=True)
    
    # 聚合逻辑：每周一至周五为一周，取首开、最高、最低、尾收、成交量和持仓量总和
    weekly_df = daily_df.resample("W-FRI").agg({
        "开盘价": "first",
        "最高价": "max",
        "最低价": "min",
        "收盘价": "last",
        "成交量": "sum",
        "持仓量": "sum",
        "动态结算价": "last"
    }).dropna()
    
    # 2. 日期筛选
    weekly_df = weekly_df[(weekly_df.index >= pd.to_datetime(start_date)) & 
                         (weekly_df.index <= pd.to_datetime(end_date))]
    weekly_df.reset_index(inplace=True)
    weekly_df.rename(columns={"index": "日期"}, inplace=True)
    return weekly_df

# 示例调用
weekly_data = futures_main_weekly_sina(symbol="V0", start_date="20240101", end_date="20251231")
print(weekly_data)

           日期     开盘价     最高价     最低价     收盘价      成交量      持仓量   动态结算价
0  2024-01-05  5880.0  5953.0  5814.0  5824.0  2755934  3516450  5843.0
1  2024-01-12  5840.0  5867.0  5754.0  5827.0  3376627  4592776  5825.0
2  2024-01-19  5835.0  5904.0  5803.0  5872.0  3078001  4140989  5856.0
3  2024-01-26  5880.0  5970.0  5823.0  5893.0  3442483  3762404  5913.0
4  2024-02-02  5898.0  5954.0  5783.0  5812.0  3252193  3855382  5818.0
..        ...     ...     ...     ...     ...      ...      ...     ...
90 2025-10-03  4894.0  4918.0  4838.0  4839.0  1246327  2147720  4858.0
91 2025-10-10  4825.0  4825.0  4719.0  4735.0  1396656  2361602  4754.0
92 2025-10-17  4730.0  4736.0  4644.0  4688.0  3456442  6243173  4706.0
93 2025-10-24  4696.0  4749.0  4670.0  4708.0  3067461  6033425  4716.0
94 2025-10-31  4708.0  4817.0  4687.0  4701.0  4286043  6040333  4729.0

[95 rows x 8 columns]


In [4]:
class FuturesDataFeed(bt.feeds.PandasData):
    """适配期货数据的Backtrader数据源"""
    params = (
        ('datetime', '日期'),
        ('open', '开盘价'),
        ('high', '最高价'),
        ('low', '最低价'),
        ('close', '收盘价'),
        ('volume', '成交量'),
        ('openinterest', '持仓量'),
    )

In [5]:
symbol_name = "棉花连续"
symbol = 'CF0'

daily_df = ak.futures_main_sina(symbol=symbol, start_date='20150101')
weekly_df = futures_main_weekly_sina(symbol=symbol, start_date="20150101")

In [6]:
print(daily_df.head())
print(daily_df.tail())

           日期    开盘价    最高价    最低价    收盘价     成交量     持仓量  动态结算价
0  2015-01-05  13040  13080  12890  13030  247762  473520  13010
1  2015-01-06  13020  13150  13015  13110  174140  478214  13090
2  2015-01-07  13120  13210  13050  13070  193080  474024  13130
3  2015-01-08  13080  13180  13035  13145  174134  479466  13125
4  2015-01-09  13165  13190  13080  13110  132142  469210  13130
              日期    开盘价    最高价    最低价    收盘价     成交量     持仓量  动态结算价
2623  2025-10-27  13550  13635  13530  13565  183256  585287  13575
2624  2025-10-28  13565  13590  13540  13565  121915  579084  13565
2625  2025-10-29  13555  13625  13520  13620  185465  578488  13585
2626  2025-10-30  13640  13675  13570  13600  211342  577586  13630
2627  2025-10-31  13605  13670  13585  13595  177483  578141  13635


In [7]:
print(weekly_df.head())
print(weekly_df.tail())

          日期      开盘价      最高价      最低价      收盘价      成交量      持仓量    动态结算价
0 2015-01-09  13040.0  13210.0  12890.0  13110.0   921258  2374434  13130.0
1 2015-01-16  13130.0  13245.0  12860.0  13125.0  1106424  2156472  13135.0
2 2015-01-23  13105.0  13190.0  12910.0  13025.0   972870  1935796  12975.0
3 2015-01-30  13005.0  13275.0  12870.0  13090.0  1233666  1904198  13150.0
4 2015-02-06  13075.0  13365.0  13040.0  13210.0  1147504  1998698  13210.0
            日期      开盘价      最高价      最低价      收盘价      成交量      持仓量    动态结算价
549 2025-10-03  13390.0  13425.0  13155.0  13215.0   560665  1066013  13245.0
550 2025-10-10  13180.0  13420.0  13180.0  13325.0   594021  1105397  13355.0
551 2025-10-17  13265.0  13380.0  13210.0  13335.0   994330  2885342  13335.0
552 2025-10-24  13360.0  13610.0  13350.0  13540.0  1157464  2969504  13550.0
553 2025-10-31  13550.0  13675.0  13520.0  13595.0   879461  2898586  13635.0


In [8]:
daily_df['日期'] = pd.to_datetime(daily_df['日期'])
daily_df = daily_df.sort_values('日期')

weekly_df['日期'] = pd.to_datetime(weekly_df['日期'])
weekly_df = weekly_df.sort_values('日期')

daily_data = FuturesDataFeed(dataname=daily_df)
weekly_data = FuturesDataFeed(dataname=weekly_df)

In [9]:
def run_backtrader_strategy():
    cerebro = bt.Cerebro()

    cerebro.adddata(daily_data, name=symbol_name)
    cerebro.adddata(weekly_data, name=symbol_name)
    cerebro.addstrategy(TripleScreenTradingSystem, symbol = 'V0')
    return cerebro.run()


In [10]:
# 每次调用都会创建新的 cerebro
# results = run_backtrader_strategy()

In [11]:
def run_backtest():
    cerebro = bt.Cerebro()
    
    # 添加策略
    cerebro.addstrategy(TripleScreenTradingSystem)
    
    # 设置初始资金
    cerebro.broker.setcash(100000.0)
    
    # 设置手续费 - 期货手续费较低
    cerebro.broker.setcommission(commission=0.0005)  # 0.05%
    cerebro.adddata(daily_data, name=f"{symbol_name}")
    cerebro.adddata(weekly_data, name=f"{symbol_name}_WEEKLY")

    
    print('初始资金: %.2f' % cerebro.broker.getvalue())
    
    # 运行回测
    cerebro.run()
    
    # 绘制图表
    cerebro.plot()
    
    print('最终资金: %.2f' % cerebro.broker.getvalue())

In [12]:
run_backtest()

初始资金: 100000.00
趋势转变! 时间: 2015-08-28, 价格: 12415.00, 类型: 下跌
趋势转变! 时间: 2016-06-03, 价格: 12470.00, 类型: 上涨
趋势转变! 时间: 2017-07-21, 价格: 15125.00, 类型: 下跌
趋势转变! 时间: 2018-01-26, 价格: 15520.00, 类型: 上涨
趋势转变! 时间: 2018-03-02, 价格: 15175.00, 类型: 下跌
趋势转变! 时间: 2018-05-11, 价格: 15670.00, 类型: 上涨
趋势转变! 时间: 2018-11-02, 价格: 15115.00, 类型: 下跌
趋势转变! 时间: 2019-05-03, 价格: 15600.00, 类型: 上涨
趋势转变! 时间: 2019-05-24, 价格: 13505.00, 类型: 下跌
趋势转变! 时间: 2020-01-03, 价格: 14015.00, 类型: 上涨
趋势转变! 时间: 2020-04-03, 价格: 10875.00, 类型: 下跌
趋势转变! 时间: 2020-09-04, 价格: 12870.00, 类型: 上涨
趋势转变! 时间: 2022-06-24, 价格: 17230.00, 类型: 下跌
趋势转变! 时间: 2023-03-10, 价格: 14340.00, 类型: 上涨
趋势转变! 时间: 2023-12-08, 价格: 15380.00, 类型: 下跌
趋势转变! 时间: 2025-07-25, 价格: 14170.00, 类型: 上涨
2015-08-28: 发出做空订单: 价格=12415.00
2015-08-31: 🔍 订单完成检查: size=-1, position=-1
2015-08-31: ✅ 空头开仓完成: 价格=12395.00, entry_bar=166, entry_price=12395.00
2015-08-31: 📋 卖出开空仓完成: 价格=12395.00, 成本=-12395.00, 手续费=6.20
2015-10-19: 空头平仓: 力量指数反转
2015-10-20: 🔍 订单完成检查: size=1, position=None
2015-10-20: ⚠️ 跳过开仓设置:

<IPython.core.display.Javascript object>

最终资金: 98124.46
